In [38]:
import numpy as np

# Constants
NUM_COURSES = 5
NUM_SECTIONS = 5
NUM_PROFESSORS = 3
NUM_DAYS = 5
NUM_TIMESLOTS = 3
NUM_ROOMS = 2
POPULATION_SIZE = 10

# Encoding lengths
course_bits = int(np.ceil(np.log2(NUM_COURSES)))
section_bits = int(np.ceil(np.log2(NUM_SECTIONS)))
professor_bits = int(np.ceil(np.log2(NUM_PROFESSORS)))
day_bits = int(np.ceil(np.log2(NUM_DAYS)))
timeslot_bits = int(np.ceil(np.log2(NUM_TIMESLOTS)))
room_bits = int(np.ceil(np.log2(NUM_ROOMS)))

# Encoding function
def encode_course(course):
  return format(course, f'0{course_bits}b')

def encode_section(section):
  return format(section, f'0{section_bits}b')

def encode_professor(professor):
  return format(professor, f'0{professor_bits}b')

def encode_day(day):
  return format(day, f'0{day_bits}b')

def encode_timeslot(timeslot):
  return format(timeslot, f'0{timeslot_bits}b')

def encode_room(room):
  return format(room, f'0{room_bits}b')

# Generate initial population
def initialize_population():
  population = []
  for _ in range(POPULATION_SIZE):
      chromosome = ''
      for _ in range(NUM_COURSES):
          course = np.random.randint(1, NUM_COURSES + 1)
          section = np.random.randint(1, NUM_SECTIONS + 1)
          professor = np.random.randint(1, NUM_PROFESSORS + 1)
          day1 = np.random.randint(1, NUM_DAYS + 1)
          day2 = (day1 % NUM_DAYS) + 1 #different days for two lectures
          timeslot1 = np.random.randint(1, NUM_TIMESLOTS + 1)
          timeslot2 = np.random.randint(1, NUM_TIMESLOTS + 1)
          room1 = np.random.randint(1, NUM_ROOMS + 1)
          room2 = np.random.randint(1, NUM_ROOMS + 1)
          room_size1 = np.random.randint(2)  # 0 classroom, 1 large hall
          room_size2 = np.random.randint(2)

          # Encoding each part and append it to the chromosome
          chromosome += encode_course(course) + \
                        encode_section(section) + \
                        encode_professor(professor) + \
                        encode_day(day1) + \
                        encode_timeslot(timeslot1) + \
                        encode_room(room1) + \
                        str(room_size1) + \
                        encode_day(day2) + \
                        encode_timeslot(timeslot2) + \
                        encode_room(room2) + \
                        str(room_size2)
      population.append(chromosome)
  return population

# Test the encoding
population = initialize_population()
print(population)


['01010010101101000110100011100011011010001111011011001101101100100101001011011010111100010110101110110101111000010110', '01110011011111010001101101011111011011001111001010011101111100101111001011010111100011010100010111000110010111100', '010001010110111100101000110001100101010110100010100101101100100011000110001010111101110110100101010010101010110', '00110101011111011001011100100101011111001011101101010010101010110110010010010001111000101110010100111000110110110101', '10010111011111011000110000101010100111001010110010010101101011000101100101011001110111001011100001101011110100111101', '01000110100011011011110101101111101101100110100010101110100110011111001000100100101100010011000011010100110100101010', '0100011010110100010110010101111001110101111110001001100101010111100001101110101110011101001001011001101111001111', '0101001100110100010111000110111101111010010110100101011001110010111101001101111001011101101001110001100011011011111', '101101101011010000111100100100100101110001111100011

In [37]:
import numpy as np

# Constants
NUM_COURSES = 5
NUM_SECTIONS = 3
NUM_PROFESSORS = 5
NUM_DAYS = 5
NUM_TIMESLOTS = 6
NUM_ROOMS = 2
POPULATION_SIZE = 10
NUM_GENERATIONS = 5
MUTATION_RATE = 0.1


CLASSROOM_CAPACITY = {'classroom': 60, 'large_hall': 120}
MAX_COURSES_PER_PROFESSOR = 3
MAX_COURSES_PER_SECTION = 5
MAX_CONSECUTIVE_DAYS = 4
BREAK_BETWEEN_CLASSES = 15  # in minutes

# Initial dictionary
initial_dictionary = {
  'Mam Zille Huma': {'courses': ['OOP'], 'sections': ['A', 'B', 'Z']},
  'Sir Usama': {'courses': ['PF'], 'sections': ['A', 'B', 'Z']},
  'Sir Saad': {'courses': ['AI'], 'sections': ['A', 'B', 'Z']},
  'Sir Bilal Dar': {'courses': ['DB'], 'sections': ['A', 'B', 'Z']},
  'Sir Owais': {'courses': ['SMD'], 'sections': ['A', 'B', 'Z']}
}

# Encoding lengths
course_bits = int(np.ceil(np.log2(NUM_COURSES)))
section_bits = int(np.ceil(np.log2(NUM_SECTIONS)))
professor_bits = int(np.ceil(np.log2(NUM_PROFESSORS)))
day_bits = int(np.ceil(np.log2(NUM_DAYS)))
timeslot_bits = int(np.ceil(np.log2(NUM_TIMESLOTS)))
room_bits = 3

# Encoding function
def encode_course(course):
  return format(course, f'0{course_bits}b')

def encode_section(section):
  return format(section, f'0{section_bits}b')

def encode_professor(professor):
  return format(professor, f'0{professor_bits}b')

def encode_day(day):
  return format(day, f'0{day_bits}b')

def encode_timeslot(timeslot):
  return format(timeslot, f'0{timeslot_bits}b')

def encode_room(room):
  return f'C-{room:03}'

def decode_timeslot(timeslot_index):
  timeslot_mapping = {
      1: '08:30 - 09:50',
      2: '10:00 - 11:20',
      3: '11:30 - 12:50',
      4: '14:30 - 15:50',
      5: '16:00 - 17:20',
      6: '17:30 - 18:50'
  }
  return timeslot_mapping[timeslot_index]

def initialize_population(num_timetables):
  population = []
  for _ in range(num_timetables):
      clashes_exist = True
      while clashes_exist:
          timetable = []
          for professor, professor_data in initial_dictionary.items():
              for course in professor_data['courses']:
                  for section in professor_data['sections']:
                      day1, day2 = select_valid_days()
                      timeslot1, timeslot2 = select_valid_timeslots(day1, day2)
                      room1, room2 = select_valid_rooms(section)
                      timetable.append((course, section, professor, day1, timeslot1, room1, day2, timeslot2, room2))
          clashes_exist = check_clashes(timetable)
      population.append(timetable)
  return population

def decode_timetable(timetable):
  decoded_timetable = []
  for course, section, professor, day1, timeslot1, room1, day2, timeslot2, room2 in timetable:
      decoded_timetable.append((course, section, professor,
                                decode_day(day1), decode_timeslot(timeslot1), encode_room(room1),
                                decode_day(day2), decode_timeslot(timeslot2), encode_room(room2)))
  return decoded_timetable

def decode_day(day):
    return ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'][day - 1]

def select_valid_days():
    day1 = np.random.randint(1, NUM_DAYS + 1)
    day2 = (day1 + 2) % NUM_DAYS + 1
    return day1, day2

def select_valid_timeslots(day1, day2):
  timeslot1 = np.random.randint(1, NUM_TIMESLOTS + 1)
  timeslot2 = (timeslot1 + 1) % NUM_TIMESLOTS + 1
  return timeslot1, timeslot2

def select_valid_rooms(section):
  room1 = np.random.randint(1, NUM_ROOMS + 1)
  room2 = np.random.randint(1, NUM_ROOMS + 1)
  return room1, room2

def check_clashes(timetable):
  professor_schedule = {}
  room_schedule = {}
  section_schedule = {}

  for course, section, professor, day1, timeslot1, room1, day2, timeslot2, room2 in timetable:
      # Check constraint Professor schedule clash
      if professor in professor_schedule:
          if (day1, timeslot1) in professor_schedule[professor] or (day2, timeslot2) in professor_schedule[professor]:
              return True
          else:
              professor_schedule[professor].add((day1, timeslot1))
              professor_schedule[professor].add((day2, timeslot2))
      else:
          professor_schedule[professor] = {(day1, timeslot1), (day2, timeslot2)}

      # Check constraint Room schedule clash
      if (section, day1, timeslot1) in room_schedule or (section, day2, timeslot2) in room_schedule:
          return True
      else:
          room_schedule[(section, day1, timeslot1)] = room1
          room_schedule[(section, day2, timeslot2)] = room2

      # Check constraint Section schedule clash
      if section in section_schedule:
          if (day1, timeslot1) in section_schedule[section] or (day2, timeslot2) in section_schedule[section]:
              return True
          else:
              section_schedule[section].add((day1, timeslot1))
              section_schedule[section].add((day2, timeslot2))
      else:
          section_schedule[section] = {(day1, timeslot1), (day2, timeslot2)}

  return False

def calculate_relative_fitness(population):
  total_fitness = sum(calculate_fitness(timetable) for timetable in population)
  if total_fitness == 0:
      return [1 / len(population)] * len(population)
  return [calculate_fitness(timetable) / total_fitness for timetable in population]

def fitness_based_selection(population, relative_fitness):
  selected_population = []
  for _ in range(len(population)):
      selected_index = np.random.choice(len(population), p=relative_fitness)
      selected_population.append(population[selected_index])
  return selected_population

def calculate_fitness(timetable):
  num_clashes = 0
  professor_schedule = {}
  room_schedule = {}
  section_schedule = {}

  for course, section, professor, day1, timeslot1, room1, day2, timeslot2, room2 in timetable:
      # Check constraint Professor schedule clash
      if professor in professor_schedule:
          if (day1, timeslot1) in professor_schedule[professor] or (day2, timeslot2) in professor_schedule[professor]:
              num_clashes += 1
          else:
              professor_schedule[professor].add((day1, timeslot1))
              professor_schedule[professor].add((day2, timeslot2))
      else:
          professor_schedule[professor] = {(day1, timeslot1), (day2, timeslot2)}

      # Check constraint Room schedule clash
      if (section, day1, timeslot1) in room_schedule or (section, day2, timeslot2) in room_schedule:
          num_clashes += 1
      else:
          room_schedule[(section, day1, timeslot1)] = room1
          room_schedule[(section, day2, timeslot2)] = room2

      # Check constraint Section schedule clash
      if section in section_schedule:
          if (day1, timeslot1) in section_schedule[section] or (day2, timeslot2) in section_schedule[section]:
              num_clashes += 1
          else:
              section_schedule[section].add((day1, timeslot1))
              section_schedule[section].add((day2, timeslot2))
      else:
          section_schedule[section] = {(day1, timeslot1), (day2, timeslot2)}
  return -num_clashes

def crossover(parent1, parent2):
  crossover_point = np.random.randint(1, len(parent1))
  child1 = parent1[:crossover_point] + parent2[crossover_point:]
  child2 = parent2[:crossover_point] + parent1[crossover_point:]
  return child1, child2

def mutate(timetable):
  mutated_timetable = []
  for course, section, professor, day1, timeslot1, room1, day2, timeslot2, room2 in timetable:
      if np.random.rand() < MUTATION_RATE:
          day1 = np.random.randint(1, NUM_DAYS + 1)
          day2 = (day1 + 2) % NUM_DAYS + 1
          timeslot1 = np.random.randint(1, NUM_TIMESLOTS + 1)
          timeslot2 = (timeslot1 + 1) % NUM_TIMESLOTS + 1
          room1 = np.random.randint(1, NUM_ROOMS + 1)
          room2 = np.random.randint(1, NUM_ROOMS + 1)
      mutated_timetable.append((course, section, professor, day1, timeslot1, room1, day2, timeslot2, room2))
  return mutated_timetable

population = initialize_population(POPULATION_SIZE)
decoded_population = [decode_timetable(timetable) for timetable in population]
print("Timetable Population:")
print("------------------------------------------------------------------------------------------------------------------------")
print("| Course | Section | Professor    | Day1    | Timeslot1     | Room1    | Day2    | Timeslot2     | Room2    |")
print("------------------------------------------------------------------------------------------------------------------------")
for timetable in decoded_population:
    for course, section, professor, day1, timeslot1, room1, day2, timeslot2, room2 in timetable:
        print(f"| {course:<6} | {section:<7} | {professor:<12} | {day1:<8} | {timeslot1:<13} | {room1:<9} | {day2:<8} | {timeslot2:<13} | {room2:<9} |")
    print("------------------------------------------------------------------------------------------------------------------------")


for generation in range(NUM_GENERATIONS):
    print(f"Generation {generation + 1}:")

    relative_fitness = calculate_relative_fitness(population)
    #fitness-based selection to select parents
    selected_population = fitness_based_selection(population, relative_fitness)
    #crossover and mutation to generate offspring
    offspring = []
    for i in range(0, len(selected_population), 2):
        parent1 = selected_population[i]
        parent2 = selected_population[i + 1]
        child1, child2 = crossover(parent1, parent2)
        child1 = mutate(child1)
        child2 = mutate(child2)
        offspring.extend([child1, child2])
    # Replace the old population with the new offspring
    population = offspring

    # Print the fittest timetable of this generation
    fittest_timetable = min(population, key=calculate_fitness)
    print("Fittest Timetable:", fittest_timetable)
    print("Fitness Score:", -calculate_fitness(fittest_timetable))


Timetable Population:
------------------------------------------------------------------------------------------------------------------------
| Course | Section | Professor    | Day1    | Timeslot1     | Room1    | Day2    | Timeslot2     | Room2    |
------------------------------------------------------------------------------------------------------------------------
| OOP    | A       | Mam Zille Huma | Thursday | 10:00 - 11:20 | C-001     | Tuesday  | 14:30 - 15:50 | C-001     |
| OOP    | B       | Mam Zille Huma | Thursday | 11:30 - 12:50 | C-001     | Tuesday  | 16:00 - 17:20 | C-002     |
| OOP    | Z       | Mam Zille Huma | Wednesday | 16:00 - 17:20 | C-002     | Monday   | 08:30 - 09:50 | C-001     |
| PF     | A       | Sir Usama    | Monday   | 08:30 - 09:50 | C-001     | Thursday | 11:30 - 12:50 | C-002     |
| PF     | B       | Sir Usama    | Friday   | 16:00 - 17:20 | C-001     | Wednesday | 08:30 - 09:50 | C-002     |
| PF     | Z       | Sir Usama    | Tuesday  | 1